# Document Retrival with Follow up Detector (DR.FUD)

In this method we use a FCN to detect wheter a question is a follow up of the previous question, meaning that the document is the same of not. If the document is the same, we use the previous answer's document for this question also.

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

## Dataset
### Dataset Description

- **mutldoc2dial_doc.json** contains the documents that are indexed by key `domain` and `doc_id` . Each document instance includes the following,

  - `doc_id`: the ID of a document;
  - `title`: the title of the document;
  - `domain`: the domain of the document;
  - `doc_text`: the text content of the document (without HTML markups);
  - `doc_html_ts`: the document content with HTML markups and the annotated spans that are indicated by `text_id` attribute, which corresponds to `id_sp`.
  - `doc_html_raw`: the document content with HTML markups and without span annotations.
  - `spans`: key-value pairs of all spans in the document, with `id_sp` as key. Each span includes the following,
    - `id_sp`: the id of a  span as noted by `text_id` in  `doc_html_ts`;
    - `start_sp`/  `end_sp`: the start/end position of the text span in `doc_text`;
    - `text_sp`: the text content of the span.
    - `id_sec`: the id of the (sub)section (e.g. `<p>`) or title (`<h2>`) that contains the span.
    - `start_sec` / `end_sec`: the start/end position of the (sub)section in `doc_text`.
    - `text_sec`: the text of the (sub)section.
    - `title`: the title of the (sub)section.
    - `parent_titles`: the parent titles of the `title`.

- **multidoc2dial_dial_train.json** and **multidoc2dial_dial_validation.json**  contain the training and dev split of dialogue data that are indexed by key `domain` . Please note: **For test split, we only include a dummy file in this version.**

  Each dialogue instance includes the following,

  - `dial_id`: the ID of a dialogue;
  - `turns`: a list of dialogue turns. Each turn includes,
    - `turn_id`: the time order of the turn;
    - `role`: either "agent" or "user";READ
    - `da`: dialogue act;
    - `references`: a list of spans with `id_sp` ,  `label` and `doc_id`. `references` is empty if a turn is for indicating previous user query not answerable or irrelevant to the document. **Note** that labels "*precondition*"/"*solution*" are fuzzy annotations that indicate whether a span is for describing a conditional context or a solution.
    - `utterance`: the human-generated utterance based on the dialogue scene.
Downloading the training dataset:

In [1]:
import json
with open('../../dataset/multidoc2dial/v1.0/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

# Constructing the Follow-up Dataset

``` history | question | is_follow_up```

is_follow_up: shows that the history's document is the same as the current question's.

In [5]:
def construct_followup_dataset(filepath):
    import json
    with open(filepath, 'r') as f:
        multidoc2dial_dial_train = json.load(f)

    historys = []
    questions = []
    labels = []
    for domain in multidoc2dial_dial_train['dial_data']:
        for dial in multidoc2dial_dial_train['dial_data'][domain]:
            prev_doc = ''
            prev_question = ''
            for turn in dial['turns']:
                if turn['role'] == "user":
                    current_question = turn['utterance']
                    historys.append(prev_question)
                    questions.append(current_question)
                    
                    current_doc = turn['references'][0]['doc_id']
                    labels.append(current_doc==prev_doc)

                    prev_doc, prev_question = current_doc, current_question
                    
    return historys, questions, labels



In [6]:
train_history, train_questions, train_labels = construct_followup_dataset('../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_train.json')
test_history, test_questions, test_labels = construct_followup_dataset('../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_validation.json')

In [7]:
import pandas as pd

train_dict_dataset = {"history":train_history, "question": train_questions, "followup": train_labels}
test_dict_dataset = {"history":test_history, "question": test_questions, "followup": test_labels}

train_df = pd.DataFrame(train_dict_dataset)
test_df = pd.DataFrame(test_dict_dataset)

In [8]:
train_df

,history,question,followup
0,,"Hello, I forgot o update my address, can you h...",False
1,"Hello, I forgot o update my address, can you h...",Can I do my DMV transactions online?,True
2,Can I do my DMV transactions online?,You've got it. Another query about DMV. What h...,False
3,You've got it. Another query about DMV. What h...,"Besides that, will I receive a notice?",True
4,"Besides that, will I receive a notice?",If you submit the affidavit?,True
...,...,...,...
23394,"By the way, who can I contact to give me infor...",What if I've fallen behind on one or more loan...,False
23395,What if I've fallen behind on one or more loan...,I have another question regarding the Military...,False
23396,I have another question regarding the Military...,something else I want to ask about FAFSA. What...,False
23397,something else I want to ask about FAFSA. What...,How can I make a payment by post?,True


In [9]:
test_df

,history,question,followup
0,,My insurance ended so what should i do,False
1,My insurance ended so what should i do,Don't do that I'll get insurance,True
2,Don't do that I'll get insurance,"I have, that is why I am here to clear that up...",True
3,"I have, that is why I am here to clear that up...",Thank you so much. After looking through these...,False
4,Thank you so much. After looking through these...,"Great. I think that I can found some bills, of...",True
...,...,...,...
4491,"If I am totally and permanently disabled, can ...",In this case I would not like,True
4492,In this case I would not like,If I am a veteran whose application for discha...,True
4493,If I am a veteran whose application for discha...,"In addition, I need to learn about PSLF. What ...",False
4494,"In addition, I need to learn about PSLF. What ...",null,True


## Encoding the sentences
We use the LaBSE which is a Language-agnostic BERT Sentence Encoder (LaBSE) is a BERT-based model trained for sentence embedding for 109 languages. The pre-training process combines masked language modeling with translation language modeling. The model is useful for getting multilingual sentence embeddings and for bi-text retrieval.

In [10]:
!pip install --quiet transformers

     |████████████████████████████████| 4.7 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 50.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 50.4 MB/s eta 0:00:01


In [6]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import torch
from torch.nn.functional import normalize

from tqdm import tqdm

In [7]:
model_name = "setu4993/LaBSE"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

### `get_embeddings`
In this method we extract the **pooler output** (Last layer hidden-state of the first token of the sequence (classification token) after further processing through the layers used for the auxiliary pretraining task. E.g. for BERT-family of models, this returns the classification token after processing through a linear layer and a tanh activation function. The linear layer weights are trained from the next sentence prediction (classification) objective during pretraining).

In [9]:
def get_embeddings(sentece):
    """
    Return embeddings based on encoder model

    :param sentence: input sentence(s)
    :type sentence: str or list of strs
    :return: embeddings
    """
    tokenized = tokenizer(sentece,
                                return_tensors="pt",
                                padding=True)
    with torch.no_grad():
        embeddings = model(**tokenized)
    
    return np.squeeze(np.array(embeddings.pooler_output))

## Calculating the IDF for each token

In [ ]:
words_idf_file = 'IDFs.pkl'
N_doc = len(doc_texts_train)

if not os.path.exists(words_idf_file):
    # First getting all distinct words in all documents
    words = set()
    doc_texts_train_tokenized = []
    for doc in tqdm(doc_texts_train, desc="getting all words from documents"):
        tokenized_doc = [s.lower() for s in tokenizer.tokenize(doc)]
        doc_texts_train_tokenized.append(tokenized_doc) 
        words = set(tokenized_doc).union(words)

    # calculating each word IDF
    words2IDF = {}
    for word in tqdm(words, desc="calculating words IDF scores"):
        n_word = 0
        for doc in doc_texts_train_tokenized:
            if word in doc:
                n_word += 1
        words2IDF[word] = np.log(N_doc / (n_word + 1))

    with open(words_idf_file, 'wb') as f:
        pickle.dump(words2IDF, f)

else:
    with open(words_idf_file, 'rb') as f:
        words2IDF = pickle.load(f)

In [ ]:
len(words2IDF)

In [ ]:
def calc_idf_score(sentence):
    """
    Calculate the mean idf score for given sentence.
    (used to understand the contribution of the knowledge of each question
    questions with high frequent words are meaningless and we can ignore them
    roughly, which is done by this score.)

    :param sentence: input sentence
    :type sentence: str
    :return: mean idf score of sentence token
    """
    tokenzied_sentence = [s.lower() for s in tokenizer.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    return score / len(tokenzied_sentence)

## Methods

## Test
In the test dataset we just picked ones with **user** turn.

In [21]:
def construct_test_set(filepath=None):
    import json
    with open('../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_train.json', 'r') as f:
        multidoc2dial_dial_train = json.load(f)

    doc_sentence_test = []
    doc_label_test = []
    for doc_idx1 in multidoc2dial_dial_train['dial_data']:
        for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
            for turns in dial['turns']:
                if turns['role'] == "user":
                    doc_sentence_test.append(turns['utterance'])
                    doc_label_test.append(turns['references'][0]['doc_id'])
    return doc_sentence_test, doc_label_test


def test_with_predictor(predictor, test_x, test_y, test_set_ratio=10):
    correct = 0
    total = 0
    counter = 0
    for x,y in tqdm(zip(test_x, test_y), desc="iterating over test set"):
        counter += 1
        if counter % test_set_ratio == 0:
            x_embed = get_embeddings(x).reshape(1, -1)
            predicted = predictor.predict(x_embed)
            correct += predicted==y
            total += 1
    return correct/len(test_y)


In [17]:
test_x, test_y = construct_test_set()

In [ ]:
# at 1
acc_at_1 = test_with_predictor(doc_predictor_1, test_x, test_y)

iterating over test set: 21210it [10:58, 32.00it/s]

## Results

At last we have resutls as follows:


| Method | @1 | @5 | @10 | @50 | @100 | MRR (mean, var) |
|:------:|:------:|:------:|:-------:|:-------:|:--------:|:---:|
| IDF - vanilla | 13% | 30% | 39% | 64% | 83% | (0.22, 0.11) |
| IDF - power-order | 15% | 31% | 41% | 65% | 83% | (0.23, 0.12) |
| IDF - power-order (softmax) | 10.7% | 23% | 31% | 57.6% | 78% | (0.18, 0.09) |
| IDF - self-attention | 13.9% | 29% | 38% | 62% | 82% | (0.22, 0.11) |
| **DR. TEIT** | **61.6%** | **86%** | **91%** | **96%** | **98%** | **(0.72, 0.13)** |

It shows that title informations were not enough for document retrieval.

# drafts

In [ ]:
tfidf_wm.shape

In [ ]:
answers = tfidfVectorizer.transform(["Original Card for a Foreign Born U.S. Citizen Adult",
                                     "Hello world from far beyound!"]).todense()
query = tfidfVectorizer.transform(["Hello!"]).todense()

In [ ]:
print(answers.shape, query.shape)

In [ ]:
import numpy as np
answers_sim = np.squeeze(np.asarray(tfidf_wm @ answers.T))
query_sim = np.squeeze(np.asarray(tfidf_wm @ query.T))

In [ ]:
print(answers_sim.shape, query_sim.shape)

In [ ]:
list(map(lambda x: np.dot(x, query_sim) /
        (np.linalg.norm(query_sim) * np.linalg.norm(x)),
        answers_sim.T))

In [ ]:
from transformers import AutoTokenizer, AutoModel, T5Tokenizer, T5EncoderModel
import torch

model_name = ["t5-small", "bert-base-uncased"][0]

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5EncoderModel.from_pretrained("t5-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
print("inputs", inputs)

print("last_hidden_states", last_hidden_states.shape)

# pooler = outputs.pooler_output
# print("pooler",pooler.shape)
# with torch.no_grad():
#     print(np.squeeze(np.array(pooler)).shape)

In [ ]:
X = [[0], [1], [2], [3]]
y = ['ali', 'ali', 'reza', 'reza']
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)
print(neigh.predict([[1.1]]))
print(neigh.predict_proba([[0.9]]))